In [2]:
import numpy as np
import scipy as sp
from numpy import ndarray, nan

import cython

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm

from IPython.display import display, Audio

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext cython

In [28]:
pwd

'/Users/bushj/projects/DUET/duet_offline'

In [ ]:
#%%cython -f -c=-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -c=-O3 -c=-ffast-math -lDUET -c=-DDUET_OFFLINE_PREPROCESS=1 -L. -I. -c=-Wl,-rpath,/Users/bushj/projects/DUET/duet_offline --verbose

In [48]:
%%cython -c=-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -c=-O3 -c=-ffast-math -lDUET -c=-DDUET_OFFLINE_PREPROCESS=1 -I. -L/Users/bushj/.cache/ipython/cython
#cython: boundscheck=False
cimport numpy as np
import numpy as np
from libc.stdint cimport int16_t

cdef extern from "duet.h":
    int duet_init(
        int window_size, # size of the STFT window, must be a power of 2 between 8 and 8192 (inclusive); e.g. 256
        int n_samples,   # number of audio samples to use for identification, must be a multiple of window_size/2; e.g. 1536
        float p,         # symmetric attenuation estimator value weight; e.g. 1.0f
        float q,         # delay estimator value weight; e.g. 0.0f
        float point_threshold # point threshold for filtering; e.g. 0.5f
    ) nogil
    void duet_deinit() nogil
    void duet_reset() nogil
    const float complex* duet_process_audio_frame(const int16_t * const frame, int* n_sources) nogil
    int duet_get_n_channels() nogil
    int duet_get_n_freq() nogil
    int duet_get_n_time() nogil
    int duet_get_audio_frame_size() nogil

def init(
    window_size: int = 256,
    n_samples: int = 1536,
    p: float = 1.0,
    q: float = 0.0,
    point_threshold: float = 0.5
) -> None:
    """
    Initializes the Duet audio processing library.
    This function must be called before using any other functions in the library.
    It initializes the FFT library, precomputes numerous values, and sets up the
    necessary parameters for the DUET algorithm along with allocating memory for
    various buffers.

    Parameters
    ----------
    window_size : int
        Size of the STFT window, must be a power of 2 between 8 and 8192 (inclusive)
    n_samples : int
        Number of audio samples to use for identification, must be a multiple of window_size/2
    p : float
        Symmetric attenuation estimator value weight
    q : float
        Delay estimator value weight
    point_threshold : float
        Point threshold for filtering
    """
    cdef int result = duet_init(window_size, n_samples, p, q, point_threshold)
    if result != 0:
        raise ValueError(f"duet_init failed with error code {result}")

def deinit() -> None:
    """
    Deinitialize the DUET audio processing library. This frees any resources
    that were allocated by duet_init() and resets the state of the library.

    This must be called to change the DUET parameters.
    """
    duet_deinit()

def reset() -> None:
    """
    Reset the DUET state for completely new, unrelated, audio. Does not
    deallocate or deinitialize any buffers or precomputed values.

    Typically this is called when switching to a new audio source but while
    keeping the same DUET parameters.
    """
    duet_reset()

def get_n_channels() -> int:
    """Returns the number of audio channels. Hardcoded to 2 for stereo audio."""
    return duet_get_n_channels()
def get_n_freq() -> int:
    """Returns the number of frequency bins in the STFT. Equal to `window_size / 2`."""
    return duet_get_n_freq()
def get_n_time() -> int:
    """Returns the number of active audio frames used for identification. Equal to `n_samples / (window_size / 2) + 1`."""
    return duet_get_n_time()
def get_audio_frame_size() -> int:
    """Returns the audio frame size. Equal to `window_size / 2 * decimation` (where `decimation` is the downsampling factor, e.g. 3)."""
    return duet_get_audio_frame_size()

def process_audio_frame(np.ndarray[int16_t, ndim=1, mode='c'] frame) -> np.ndarray:
    """
    Add the new audio frame to the existing audio buffer and process it with
    DUET. The new audio frame is interleaved channel data with audio frame size
    samples for each channel (see `duet_get_audio_frame_size()`). Overall size
    must be exactly `n_channels * audio_frame_size` samples.

    This returns the demixed sources of shape (n_sources, N_FREQ, N_TIME) i.e.
    for each source there is a mono-spectrogram across all active audio frames.

    Due to incremental processing, it is recommended to call this N_TIME times
    before attempting to interpret the results.

    Parameters
    ----------
    frame : np.ndarray
        A 1D numpy array of int16 audio samples. The length of the array must be
        equal to the audio frame size times the number of channels.

    Returns
    -------
    demixed : np.ndarray
        A 3D numpy array of shape (n_sources, N_FREQ, N_TIME) containing the
        demixed audio sources as complex-valued spectrograms.
    """
    # validate input
    if frame.size != duet_get_audio_frame_size() * duet_get_n_channels():
        raise ValueError(f"Input frame must have exactly {duet_get_audio_frame_size() * duet_get_n_channels()} samples")

    # call C function
    cdef int n_sources = 0, n_freq = duet_get_n_freq(), n_time = duet_get_n_time()
    cdef const float complex* c_out
    with nogil: c_out = duet_process_audio_frame(&frame[0], &n_sources)

    # convert to numpy array
    if c_out == NULL: raise RuntimeError("duet_process_audio_frame failed")
    out = np.empty((n_sources, n_freq, n_time), dtype=np.complex64)
    if n_sources <= 0: return out
    np.copyto(out, np.frombuffer(<char*>c_out, dtype=np.complex64, count=n_sources * n_freq * n_time).reshape((n_sources, n_freq, n_time)))
    return out


Content of stderr:
/Users/bushj/.cache/ipython/cython/_cython_magic_7073f36e7683dc460e7f4a48b444fa7f3ed5b655.c:8825:26: warning: code will never be executed [-Wunreachable-code]
 8825 |                 module = PyImport_ImportModuleLevelObject(
      |                          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.
/Users/bushj/.cache/ipython/cython/_cython_magic_7073f36e7683dc460e7f4a48b444fa7f3ed5b655.c:8825:26: warning: code will never be executed [-Wunreachable-code]
 8825 |                 module = PyImport_ImportModuleLevelObject(
      |                          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.

ImportError: dlopen(/Users/bushj/.cache/ipython/cython/_cython_magic_7073f36e7683dc460e7f4a48b444fa7f3ed5b655.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libDUET.dylib
  Referenced from: <71B48809-F1BB-3118-9A00-BCCE966A584A> /Users/bushj/.cache/ipython/cython/_cython_magic_7073f36e7683dc460e7f4a48b444fa7f3ed5b655.cpython-310-darwin.so
  Reason: no LC_RPATH's found

In [15]:
%%cython?

Docstring:
::

  %cython [-a] [--annotate-fullc] [-+] [-3] [-2] [-f] [-c COMPILE_ARGS]
              [--link-args LINK_ARGS] [-l LIB] [-n NAME] [-L dir] [-I INCLUDE]
              [-S SRC] [--pgo] [--verbose]

Compile and import everything from a Cython code cell.

The contents of the cell are written to a `.pyx` file in the
directory `IPYTHONDIR/cython` using a filename with the hash of the
code. This file is then cythonized and compiled. The resulting module
is imported and all of its symbols are injected into the user's
namespace. The usage is similar to that of `%%cython_pyximport` but
you don't have to pass a module name::

    %%cython
    def f(x):
        return 2.0*x

To compile OpenMP codes, pass the required  `--compile-args`
and `--link-args`.  For example with gcc::

    %%cython --compile-args=-fopenmp --link-args=-fopenmp
    ...

To enable profile guided optimisation, pass the ``--pgo`` option.
Note that the cell itself needs to take care of establishing a suitable
prof